# Multi-Sensor Atmospheric Correction in Google Earth Engine by Collection

**Description:** This script allows to do atmospheric correction for list of images of Sentinel-2 and Landsat sensors, especifically for images over coastal or oceanic areas. Some atmospheric correction settings can be modified in the *parameters.py* module to work with images over inland areas (See line 36 in that module). The script does AC automatically by providing the right satellite mission (**mission**), list of images (**imageID**), and a specific GEE Asset (**assetID**) to export processed images to your personal GEE account.<br/>
More sensors can be added by modifying the *mission_specifics.py* and *parameters.py* modules to properly work with the available collections in GEE and [Py6S](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py).<br/>

Script modified from https://github.com/samsammurphy/gee-atmcorr-S2<br/>
By Luis Lizcano-Sandoval<br/>
College of Marine Science, University of South Florida<br/>
luislizcanos@usf.edu<br/>
Created: 10/30/2020<br/>
Updated: 09/02/2021

### **Set Colab, install missing libraries, and clone github repo.**


In [1]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/drive


In [2]:
## Authenticate your EE account
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Z6gTXVonz2vYNvco3E4CryHhrzuot8WPqZXGr8sJMn0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgcaaO2dRSFPD5H4f6VwVe-sv8SutTsyUVlDfNeVQdTjmQ8Ef3eyiw

Successfully saved authorization token.


In [5]:
## Install some libraries:
!pip install Py6S

     |████████████████████████████████| 66 kB 2.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 47 kB 3.7 MB/s 
  Created wheel for Py6S: filename=Py6S-1.9.0-py3-none-any.whl size=76995 sha256=b01519f9bd70977d7982aa29e24f37c0eb1f68a6d018d101d2ffe68eed22cd0f
  Stored in directory: /root/.cache/pip/wheels/12/62/93/25325c89e4debfbcfd6bdde38272d7971b7af53cc6e6d85599
Successfully built Py6S


In [6]:
## Clone github repo:
!git clone https://github.com/luislizcano/gee-atmcorr-py6s.git

Cloning into 'gee-atmcorr-py6s'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 183 (delta 90), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (183/183), 3.08 MiB | 14.80 MiB/s, done.
Resolving deltas: 100% (90/90), done.


### Import modules and initialize Earth Engine



In [8]:
import os, sys
sys.path.insert(0,'/content/gee-atmcorr-py6s')
sys.path.append('/content/gee-atmcorr-py6s/bin')
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
import mission_specifics as mn
import getBOA
import timeit

ee.Initialize()

### Earth Engine Collections
Set the collection of interest using one this specific categories:


In [ ]:
mission = 'Sentinel2'
# mission = 'Landsat8'
#mission = 'Landsat7'
# mission = 'Landsat5'

## Define user asset and folder to save output in GEE:
userAsset = 'users/lizcanosandoval/BOA/'
outputFolder = 'FL_19'

### Image ID's
Paste the list of image IDs.

In [ ]:
imageID = ['20191207T160509_20191207T160505_T17RNH',
        '20190411T160519_20190411T160516_T17RNH'] #Sentinel-2
# imageID = ['LC08_015043_20191015', 'LC08_015043_20190727'] #Landsat8
# imageID = ['LE07_015043_20191023','LE07_015043_20191108',
#           'LE07_017040_20190223','LE07_017040_20191005'] #Landsat7
# imageID = ['LT05_015043_19901015','LT05_015043_19901218','LT05_017040_19901029',
#           'LT05_017040_19901114','LT05_015043_19890521','LT05_017040_19891111'] #Landsat5

## Sort list
imageID = sorted(imageID)
print(sorted(imageID))

### Load Collection
Get the respective scenes from the collection

In [ ]:
# Load collection
collection = ee.ImageCollection(mn.eeCollection(mission)).filter(ee.Filter.inList('system:index',imageID))
firstImage = collection.first()
count = collection.size()
print('Number of images in this collection: ', count.getInfo())

#  ImageCollection to List           
col_list = collection.toList(count) #ee.List
#col_size = col_list.size().getInfo() #Python list
#print(col_size)

# Select an image for debugging
for i in range(count.getInfo()):
    image = col_list.get(i)#first()
    print('Image '+str(i)+':', image.getInfo()['properties']['system:index'])

# Date
dateString = datetime.datetime.utcfromtimestamp(firstImage.get('system:time_start').getInfo()/1000).strftime("%Y-%m-%d")
print('Date-First Image: ',dateString)

print('Mission: ', mission)

### Atmospheric Correction
Available bands available for atmospheric correction for each sensor, according to the [Py6S module](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py): <br/>
* **Sentinel2:** ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'] {Can Skip B6,B7,B8A,B9,B10},<br/>
* **Landsat8:** ['B1','B2','B3','B4','B5','B6','B7','B8','B9'] {B10,B11 - Thermal}{Can Skip B8,B9,B11},<br/>
* **Landsat7:** ['B1','B2','B3','B4','B5','B7'] {B6_VCID_1 - Thermal},<br/>
* **Landsat5:** ['B1','B2','B3','B4','B5','B7'] {B6 - Thermal},<br/>
* **Landsat4:** ['B1','B2','B3','B4','B5','B7'] {B6 - Thermal}<br/>

The respective QA band will be preserved after the correction:<br/>
* **Sentinel2:** 'QA60',<br/>
* **Landsat sensors:** 'BQA'

**NOTE:** Sentinel's B8 might show negative reflectances on coastal and oceanic areas, but not on cloudy pixels, so it does not affect cloud masking procedures. I only use bands B8, B11, B12 for cloud masking. If you need to use these bands for other purposes just check the areas of negative values are not large, otherwise I would not recommend to use them (it is up to you). The negative reflectances might be due to overestimations of aerosols at sea-level in coastal areas and suspendend particles in water. Band B10 should not be provided as a surface reflectance output, because it does not provide information on the surface but on the cirrus clouds [[Main-Knorn et al. 2017]](https://www.researchgate.net/publication/320231869_Sen2Cor_for_Sentinel-2). Landsat sensors may show a similar behaviour in infrared bands. 

In [ ]:
## Default bands of interest:
if 'Sentinel' in mission:
    bands = ['B1','B2','B3','B4','B5','B8','B11','B12'] #Sentinel-2
elif 'Landsat8' in mission:
    bands = ['B1','B2','B3','B4','B5','B6','B7'] #Landsat-8
else:
    bands = ['B1','B2','B3','B4','B5','B7'] #Landsat-7/5

Run atmospheric correction (for Sentinel, it takes ~20s per image (8 bands)).

In [ ]:
%%time
boaCollection = getBOA.forCollection(collection, mission, bands, imageID)

## Verify that each band is present in the output:
print('Output bands: ', boaCollection.first().bandNames().getInfo())

Rename the variable 'mission' if working with Sentinel-2

In [ ]:
# If working with S-2, then identify whether the sensor is Sentinel-2A or Sentinel-2B, and rename the variable 'mission'
if 'Sentinel2' == mission:
    mission = str(firstImage.getInfo()['properties']['SPACECRAFT_NAME'])

### Display results
Shows an image as example.

In [ ]:
from IPython.display import display, Image

boaImage = boaCollection.first()
region = boaImage.geometry().buffer(5000).bounds().getInfo()['coordinates']

# RGB Bands
channels = []
if 'Sentinel' in mission or 'Landsat8' == mission:
    channels = ['B4','B3','B2'] #For Sentinel & Landsat8
else:
    channels = ['B3','B2','B1'] #For Landsat7-5-4

# Display images:
original = Image(url=mn.TOA(firstImage,mission).select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25
    }))

corrected = Image(url=boaImage.select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25,
    'gamma':1.8
    }))

display(original, corrected)

### Export to Asset

NOTE: 
* Be aware that each band will be resampled at the scale used to export the image. This will impact the size of the exported file.
* A Sentinel-2 image with 8 bands at 10m res each can occupy ~1.5 gb.
* A Sentinel-2 image with 8 bands can take ~4-8 min to ingest.

In [ ]:
## Run the export

toaList = collection.toList(collection.size())
boaList = boaCollection.toList(boaCollection.size())
boaSize = boaList.size().getInfo()

print('Wait for submission')

for i in range(boaSize):
    ## Copy properties from the original image
    toa = ee.Image(toaList.get(i))
    image = ee.Image(boaList.get(i))
    
    ## Set the scale properly
    scale = []
    sat = []
    tile = []
    if 'Sentinel' in mission:
        sat = 'Sentinel'
        scale = 10 #For Sentinel
        tile = (image.getInfo()['properties']['MGRS_TILE'])
    else:
        sat = 'Landsat'
        scale = 30 #For Landsat 
        tile = str(image.getInfo()['properties']['WRS_PATH'])+str(image.getInfo()['properties']['WRS_ROW'])
    
    ## Get properties from each image
    imgID = toa.getInfo()['properties']['system:index'] #The solution is copying the id from the original collection.
    date = datetime.datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000).strftime("%Y-%m-%d")
    tileID = tile
    
    ## Set some properties for export
    output = image.set({'satellite': mission,
                   'tile_id': str(tileID),
                   'file_id': imgID,                                               
                   'date': date,
                   'generator': 'Lizcano-Sandoval',
                        })

    ## Define YOUR assetID. (This do not create folders, you need to create them manually)
    assetID = userAsset+sat+'/'+outputFolder+'/' ##This goes to an ImageCollection folder
    fileName = str(tileID) + '_' + imgID +'_BOA'
    path = assetID + fileName

    ## Batch Export to Assets
    ee.batch.Export.image.toAsset(\
        image = ee.Image(output),                                                    
        description = 'BOA_'+imgID,
        assetId = path,
        region = image.geometry().buffer(10),                                      
        maxPixels = 1e9,
        scale = scale).start()
    print('Image '+str(i+1)+': '+imgID+' submitted...')
print('All images submitted!')